In [214]:
import random
from glob import glob
from time import time
from typing import Union

import pandas as pd
import numpy as np
from ord_data_load import ORD_PATH, ORD_REPO_PATH

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#to disable warnings
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


import os
import multiprocessing as mp
import gzip
from google import protobuf

from ord_schema import message_helpers
from ord_schema.proto import dataset_pb2
from ord_schema.proto import reaction_pb2

from time import time
from ord_data_load import load_dataset, filter_uspto_filenames

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
# uspto_filenames = []
# N = len(glob(f'{ORD_REPO_PATH}/data/*/*.pb.gz'))
#
# start = time()
# for i, pb in enumerate(glob(f'{ORD_REPO_PATH}/data/*/*.pb.gz'), 1):
#     print(f"{i:3d} / {N}: parsing dataset {time() - start:.1f}s", end="\r")
#     dataset = load_dataset(pb)
#     if "uspto" in dataset.name:
#         uspto_filenames.append(pb)
#
# len(uspto_filenames)

print("515 / 515: parsing dataset 134.3s")

515 / 515: parsing dataset 134.3s


In [268]:
%%time

if os.path.exists(f"{ORD_PATH}/uspto_files.csv"):
    print("uspto_files.csv already exists ... loading")
    uspto_files = pd.read_csv(f"{ORD_PATH}/uspto_files.csv").squeeze()
else:
    print("extracting uspto filenames ... ", end="")
    n_cores = 24
    with mp.Pool(n_cores) as p:
        uspto_filenames = p.map(filter_uspto_filenames, glob(f'{ORD_REPO_PATH}/data/*/*.pb.gz'))

    # save results
    uspto_files = pd.Series(uspto_filenames).dropna()
    uspto_files.to_csv(f"{ORD_PATH}/uspto_files.csv", index=False, header=False)
    print("saved to uspto_files.csv")

uspto_files.csv already exists ... loading
CPU times: total: 0 ns
Wall time: 5 ms


# Create parsing method for USPTO compounds

In [219]:
uspto_files = pd.read_csv(f"{ORD_PATH}/uspto_files.csv").squeeze()

pb = uspto_files.sample().iat[0]
pb

'./ord-data/data\\ac\\ord_dataset-ac1c7aa04d6c4e588e723e3e05721681.pb.gz'

In [220]:
%%time
pb = glob(f'{ORD_REPO_PATH}/data/*/*c3c1091f873b4f40827973a6f1f9b685.pb.gz')[0]
dataset = load_dataset(pb)

CPU times: total: 1.11 s
Wall time: 1.11 s


In [221]:
print(dataset.name)
print(dataset.description)
print(len(dataset.reactions))

uspto-grants-2014_09
CML filenames: I20140902.xml,I20140909.xml,I20140916.xml,I20140923.xml,I20140930.xml
17639


In [222]:
def get_rxn_role(val):
    return reaction_pb2.ReactionRole.ReactionRoleType.Name(val)

get_rxn_role(1)

'REACTANT'

In [223]:
%%time
rxn = random.choice(dataset.reactions)
for cmpd in message_helpers.find_submessages(rxn, reaction_pb2.Compound):
    for i in cmpd.identifiers:
        if i.type == reaction_pb2.CompoundIdentifier.NAME:
            print("name:", i.value)
        if i.type == reaction_pb2.CompoundIdentifier.SMILES:
            print("smiles: ", i.value)
    print("role:", get_rxn_role(cmpd.reaction_role))
    print("=========================================================================")
for cmpd in message_helpers.find_submessages(rxn, reaction_pb2.ProductCompound):
    for i in cmpd.identifiers:
        if i.type == reaction_pb2.CompoundIdentifier.NAME:
            print("name:", i.value)
        if i.type == reaction_pb2.CompoundIdentifier.SMILES:
            print("smiles: ", i.value)
    print("role:", get_rxn_role(cmpd.reaction_role))
    print("=========================================================================")
print(rxn.notes)

name: 2-(2,6-difluorophenoxy)-5,5,6,6-tetramethyl-5,6-dihydro-1,4,3-oxathiazine 4,4-dioxide
smiles:  FC1=C(OC=2OC(C(S(N2)(=O)=O)(C)C)(C)C)C(=CC=C1)F
role: REACTANT
name: (S)-1-phenylethylamine
smiles:  C1(=CC=CC=C1)[C@H](C)N
role: REACTANT
name: ethyl acetate
smiles:  C(C)(=O)OCC
role: WORKUP
name: MgSO4
smiles:  [O-]S(=O)(=O)[O-].[Mg+2]
role: WORKUP
name: product
name: ((S)-1-Phenylethyl)-(5,5,6,6-tetramethyl-4,4-dioxo-5,6-dihydro-4H-4lambda6-[1,4,3]oxathiazin-2-yl)amine
smiles:  C1(=CC=CC=C1)[C@H](C)NC=1OC(C(S(N1)(=O)=O)(C)C)(C)C
role: PRODUCT
procedure_details: "A mixture of 194 mg of 2-(2,6-difluorophenoxy)-5,5,6,6-tetramethyl-5,6-dihydro-1,4,3-oxathiazine 4,4-dioxide and 737 mg of (S)-1-phenylethylamine was stirred at room temperature for 16 hours and then purified in a purification laboratory by means of preparative HPLC. The combined product-containing fractions were alkalized with 25% aqueous ammonia solution and extracted three times with 30 ml of ethyl acetate, and the combin

In [224]:
%%time
pb = random.choice(uspto_files)
# pb = glob(f'{ORD_REPO_PATH}/data/*/*c3c1091f873b4f40827973a6f1f9b685.pb.gz')[0]

dataset = load_dataset(pb)
cmpd_count = 0
for rxn in dataset.reactions:
    cmpd_count += len(message_helpers.find_submessages(rxn, reaction_pb2.Compound))
    cmpd_count += len(message_helpers.find_submessages(rxn, reaction_pb2.ProductCompound))

print("Total reactions parsed:", len(dataset.reactions))
print("Total compounds parsed:", cmpd_count)
print("Compounds per rxn     :", cmpd_count / len(dataset.reactions))

Total reactions parsed: 4593
Total compounds parsed: 36656
Compounds per rxn     : 7.980840409318528
CPU times: total: 2.02 s
Wall time: 2.02 s


In [225]:
%%time
cmpd_count = 0
max_len = 0
for rxn in dataset.reactions:
    for cmpd in message_helpers.find_submessages(rxn, reaction_pb2.Compound):
    # for cmpd in message_helpers.find_submessages(rxn, reaction_pb2.ProductCompound):
        cmpd_count += 1

        names = []
        for i in cmpd.identifiers:
            if i.type == reaction_pb2.CompoundIdentifier.NAME:
                names.append(i.value)
        if len(names) > 1:
            # pass
            # print(names)
            if len(names) > max_len:
                max_len = len(names)


print("=================================================")
print("Fraction of multinames:", len(names) / cmpd_count)
print("max len:", max_len)

Fraction of multinames: 3.139323161926289e-05
max len: 2
CPU times: total: 984 ms
Wall time: 977 ms


In [226]:
get_rxn_role(2)

'REAGENT'

In [379]:
"""
    Numpy array of compounds with two indexes [0 .. total_cmpd_numer, field_idx]
    - first idx:
        Compound index
    - second idx:
        Corresponding field index
        ['second name', 'name', 'smiles', 'role', 'rxn_id']
        arr[i, 0] - (str/optional), trivial name or compound label in the patent
        arr[i, 1] - (str), systematic name
        arr[i, 2] - (str), smiles
        arr[i, 3] - (int), reaction role enum from reaction_pb2.ReactionRole.ReactionRoleType,
                           e.g. "REACTANT" - 1, "SOLVENT" - 3, "CATALYST" - 4, "PRODUCT" - 8
        arr[i, 4] - (str), reaction_id, e.g. "ord-43d5b7a6265d46a0ab8a7e2b2db5ad33"

"""

def parse_compound(arr: np.array,
                   idx: int,
                   rxn: reaction_pb2.Reaction,
                   cmpd: Union[reaction_pb2.Compound, reaction_pb2.ProductCompound]
                   ):
    names = []
    for i in cmpd.identifiers:
        if i.type == reaction_pb2.CompoundIdentifier.NAME:
            names.append(i.value)
        if i.type == reaction_pb2.CompoundIdentifier.SMILES:
            arr[idx, 2] = i.value

    if len(names) == 1:
        arr[idx, 1] = names[0]
    else:
        arr[idx, 0] = names[0]
        arr[idx, 1] = names[1]

    arr[idx, 3] = cmpd.reaction_role
    arr[idx, 4] = rxn.reaction_id

def parse_product(arr: np.array,
                   idx: int,
                   rxn: reaction_pb2.Reaction,
                   cmpd: Union[reaction_pb2.Compound, reaction_pb2.ProductCompound]
                   ):

    parse_compound(arr, idx, rxn, cmpd)
    # products sometimes have UNDEFINED rxn_role
    if arr[idx, 3] == 0: # UNDEFINED
        arr[idx, 3] = 8 # PRODUCT


start = time()
pb = glob(f'{ORD_REPO_PATH}/data/*/*c3c1091f873b4f40827973a6f1f9b685.pb.gz')[0]
# pb = random.choice(uspto_files)
dataset = load_dataset(pb)
print(f"Dataset {dataset.name} loaded in {time() - start:.2f}s", )

start = time()


def parse_dataset(dataset: dataset_pb2.Dataset) -> np.array:
    N = len(dataset.reactions)
    arr = np.empty((N*10, 5), dtype=object) # upper limit ~10x compounds per reaction
    idx = 0

    for rxn in dataset.reactions:
        compounds = []
        products = []

        for key in rxn.inputs:
            compounds.extend(rxn.inputs[key].components)
        products.extend(rxn.outcomes[0].products)

        for cmpd in compounds:
            parse_compound(arr, idx, rxn, cmpd)
            idx += 1
        for cmpd in products:
            parse_product(arr, idx, rxn, cmpd)
            idx += 1
    arr = np.resize(arr, (idx, 5))
    return arr

arr = parse_dataset(dataset)

print(f"Dataset {dataset.name} parsed in {time() - start:.2f}s", )
print(arr.shape)
print("Total reactions parsed:", len(dataset.reactions))
print("Total compounds parsed:", len(arr))
print("Compounds per rxn     :", len(arr) / len(dataset.reactions))
# print("Max NAME len          :", np.char.str_len(reactants[:, 0]).max())
# print("Max SMILES len        :", np.char.str_len(reactants[:, 1]).max())
print(f"Size                 : {arr.nbytes / 1024 / 1024:.1f}MB")
print(f"Size per rxn         : {arr.nbytes / len(dataset.reactions):.0f}B")

Dataset uspto-grants-2014_09 loaded in 1.05s
Dataset uspto-grants-2014_09 parsed in 1.44s
(99416, 5)
Total reactions parsed: 17639
Total compounds parsed: 99416
Compounds per rxn     : 5.63614717387607
Size                 : 3.8MB
Size per rxn         : 225B


In [380]:
arr

array([[None, 'acetyl chloride', 'C(C)(=O)Cl', 1,
        'ord-43d5b7a6265d46a0ab8a7e2b2db5ad33'],
       [None, 'Hydrochloric acid', 'Cl', 1,
        'ord-43d5b7a6265d46a0ab8a7e2b2db5ad33'],
       [None, '3-amino-2-naphthol', 'NC=1C(=CC2=CC=CC=C2C1)O', 1,
        'ord-43d5b7a6265d46a0ab8a7e2b2db5ad33'],
       ...,
       [None, 'trifluoroacetic acid', 'FC(C(=O)O)(F)F', 1,
        'ord-470523d3fae643daa127a94a8bc31299'],
       [None, 'dichloromethane', 'ClCCl', 3,
        'ord-470523d3fae643daa127a94a8bc31299'],
       [None,
        '(1R,3aS,5aR,5bR,7aR,11aR,11bR,13aR,13bR)-3a-amino-5a,5b,8,8,11a-pentamethyl-1-(prop-1-en-2-yl)-2,3,3a,4,5,5a,5b,6,7,7a,8,11,11a,11b,12,13,13a,13b-octadecahydro-1H-cyclopenta[a]chrysen-9-yl trifluoromethanesulfonate',
        'FC(S(=O)(=O)OC=1C([C@@H]2CC[C@]3([C@@]4(CC[C@@]5([C@@H]([C@H]4CC[C@@H]3[C@]2(CC1)C)[C@@H](CC5)C(=C)C)N)C)C)(C)C)(F)F',
        8, 'ord-470523d3fae643daa127a94a8bc31299']], dtype=object)

In [381]:
arr[~np.equal(arr[:, 0], None)]

array([['D3', 'cholecalciferol',
        'C[C@H](CCCC(C)C)[C@H]1CC[C@@H]\\2[C@@]1(CCC/C2=C\\C=C/3\\C[C@H](CCC3=C)O)C',
        1, 'ord-a2dadd22025a409987662a0f64ebd88e'],
       ['product',
        'N-allyl-N-methyl-3-[(3-methyl-4-nitrophenyl)amino]-N-{3-[(3-methyl-4-nitrophenyl)amino]propyl}-1-propanaminium p-toluene sulfonate',
        'C1(=CC=C(C=C1)S(=O)(=O)[O-])C.C(C=C)[N+](CCCNC1=CC(=C(C=C1)[N+](=O)[O-])C)(CCCNC1=CC(=C(C=C1)[N+](=O)[O-])C)C',
        8, 'ord-36d795d1da95405ea6c75daa91c54980'],
       ['title compound', 'Auristatin F hydroxypropylamide',
        'CC[C@H](C)[C@@H]([C@@H](CC(=O)N1CCC[C@H]1[C@@H]([C@@H](C)C(=O)N[C@@H](CC2=CC=CC=C2)C(=O)O)OC)OC)N(C)C(=O)[C@H](C(C)C)NC(=O)[C@H](C(C)C)N(C)C.OCCC[NH-]',
        8, 'ord-e1e02cc4069141228fd25e1f04d271b5'],
       ...,
       ['title compound',
        '4-((1R,3aS,5aR,5bR,7aR,11aS,11bR,13aR,13bR)-3a-((2-(4-(cyclopropanecarbonyl)piperazin-1-yl)ethyl)amino)-5a,5b,8,8,11a-pentamethyl-1-(prop-1-en-2-yl)-2,3,3a,4,5,5a,5b,6,7,7a,

In [385]:
arr[arr[:, 3] == 8].shape[0]

18290

In [389]:
%%time
arr[arr[:, 3] == 8]

CPU times: total: 0 ns
Wall time: 6 ms


array([[None, 'N-(3-hydroxynaphthalen-2-yl)acetamide',
        'OC=1C(=CC2=CC=CC=C2C1)NC(C)=O', 8,
        'ord-43d5b7a6265d46a0ab8a7e2b2db5ad33'],
       [None, 'N-(3-propoxynaphthalen-2-yl)acetamide',
        'C(CC)OC=1C(=CC2=CC=CC=C2C1)NC(C)=O', 8,
        'ord-1e9795495fcf41dabac210930fb8b1eb'],
       [None, '3-propoxynaphthalen-2-ylamine',
        'C(CC)OC=1C(=CC2=CC=CC=C2C1)N', 8,
        'ord-531862ea724b4025a6820c232ec276a5'],
       ...,
       [None,
        '(1R,3aS,5aR,5bR,7aR,9S,11aR,11bR,13aR,13bR)-3a-isocyanato-5a,5b,8,8,11a-pentamethyl-1-(prop-1-en-2-yl)icosahydro-1H-cyclopenta[a]chrysen-9-ol',
        'N(=C=O)[C@]12[C@@H]([C@H]3CC[C@@H]4[C@]5(CC[C@@H](C([C@@H]5CC[C@]4([C@@]3(CC1)C)C)(C)C)O)C)[C@@H](CC2)C(=C)C',
        8, 'ord-3884eaf8b34b40be8c2bd3129a54742e'],
       [None,
        'tert-butyl ((1R,3aS,5aR,5bR,7aR,11aR,11bR,13aR,13bR)-5a,5b,8,8,11a-pentamethyl-9-oxo-1-(prop-1-en-2-yl)icosahydro-1H-cyclopenta[a]chrysen-3a-yl)carbamate',
        'C[C@]12CC[C@@]3([C@@H

In [390]:
%%time

u, c = np.unique(arr[arr[:, 3] == 8][:, 4], return_counts=True)
u[c > 1]

CPU times: total: 15.6 ms
Wall time: 23 ms


array(['ord-0099398f1d1a4ecea0b0514be075a909',
       'ord-00a671d0dddb4b06899ba5c0259a586d',
       'ord-00c409ed36c1428abc020e6a9b357b4e',
       'ord-00d15985189c4c47b04522dc6b15de83',
       'ord-010b17c1f572475b8b606553da5c7c93',
       'ord-012f4d44787242e7a8e70cab32f230d5',
       'ord-017020620353467182ff82ac45df7da9',
       'ord-01c13c1fe8664388bf3b8405f5196636',
       'ord-01f4839d1f2846ee93e0257eb0d561d9',
       'ord-024dae92cff1471da05371e4cf2a3653',
       'ord-02f660f5831a40ca9a72dcea729a72c7',
       'ord-032bc698151a4a73af28dc680686db87',
       'ord-0349029a989b4c78add46a8315d55694',
       'ord-03d238cb8bc84486bd2a43f4fafcb5b6',
       'ord-03d7974576cd4c959a67de3464264e0b',
       'ord-03fd1c1607454b2a81c292209e67120e',
       'ord-0421e20749bc42b0aa0fd3b2beaf0007',
       'ord-0454cfcfff4f4525ae5c8632df657eb0',
       'ord-050e667941c042f1a9a63cc20ed78f75',
       'ord-0627357539d24bd09093b16f487d80df',
       'ord-06da7730e71a485db5d475cb2e417b1d',
       'ord-0

In [386]:
np.unique(arr[:, 4]).size

17639

In [239]:
names_unique = np.unique(arr[~np.equal(arr[:, 1], None)][:, 1], return_counts=True)
# np.unique(arr[:, 1])

In [240]:
names_unique[0][np.argsort(names_unique[1])][-20:]

array(['acetonitrile', 'acetic acid', 'dioxane', 'toluene', 'Water',
       'potassium carbonate', 'CH2Cl2', 'EtOAc', 'MeOH', 'ethanol',
       'triethylamine', 'DCM', 'tetrahydrofuran', 'HCl', 'methanol',
       'ethyl acetate', 'dichloromethane', 'DMF', 'THF', 'water'],
      dtype=object)

In [241]:
%%time
len_vect = np.vectorize(lambda x: len(str(x)))
len_vect(arr[:, 1]).max()

CPU times: total: 31.2 ms
Wall time: 28 ms


320

# Parse whole USPTO dataset to numpy array of compounds

In [391]:
import numpy as np
import pandas as pd
from ord_data_load import pb2_to_numpy_cmpd, ORD_PATH

uspto_files = pd.read_csv(f"{ORD_PATH}/uspto_files.csv").squeeze()

In [392]:
%%time
n_cores = 24

if __name__ == '__main__':
    with mp.Pool(n_cores) as p:
        res = p.map(pb2_to_numpy_cmpd, uspto_files)
len(res)

CPU times: total: 9.2 s
Wall time: 27.3 s


488

In [393]:
cmpd_np = np.vstack(res)
cmpd_np

array([[None, 'sulfuric acid', 'S(O)(O)(=O)=O', 1,
        'ord-f05182efc7494917864a6ea459070b88'],
       [None, 'phosphate rock',
        '[O-]P(=O)([O-])[O-].[O-]P(=O)([O-])[O-].[O-]P(=O)([O-])[O-].[F-].[Ca+2].[Ca+2].[Ca+2].[Ca+2].[Ca+2]',
        1, 'ord-f05182efc7494917864a6ea459070b88'],
       [None, 'phosphoric acid', 'P(O)(O)(O)=O', 1,
        'ord-f05182efc7494917864a6ea459070b88'],
       ...,
       [None, 'NaOMe', 'C[O-].[Na+]', 1,
        'ord-2b0f97b346414e2083183fb1d64385e2'],
       [None, 'MeOH', 'CO', 3, 'ord-2b0f97b346414e2083183fb1d64385e2'],
       [None,
        'N-hexyl-6-hydroxy-2,5,7,8-tetramethylchroman-2-carboxamide',
        'C(CCCCC)NC(=O)C1(OC2=C(C(=C(C(=C2CC1)C)O)C)C)C', 8,
        'ord-2b0f97b346414e2083183fb1d64385e2']], dtype=object)

In [397]:
print(f"{cmpd_np.nbytes/1024/1024:.1f}MB")

378.1MB


In [398]:
%%time

np.save(f'{ORD_PATH}/cmpd_np.npy', cmpd_np)

CPU times: total: 13.1 s
Wall time: 13.9 s


In [399]:
%%time
cmpd_np = np.load(f'{ORD_PATH}/cmpd_np.npy', allow_pickle=True)

CPU times: total: 6.86 s
Wall time: 6.88 s


In [400]:
%%time
df = pd.DataFrame(cmpd_np, columns=['trivial', 'name', 'smiles', 'rxn_role', 'rxn_id'])
# df['rxn_role'] = df['rxn_role'].astype("category")
df.set_index('rxn_id', inplace=True)
df.sort_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9912637 entries, ord-000002a8654f41fc85d99dbf87521aa1 to ord-fffffe4d14804a56bd6d6f6d64d98144
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   trivial   object
 1   name      object
 2   smiles    object
 3   rxn_role  object
dtypes: object(4)
memory usage: 378.1+ MB
CPU times: total: 20.7 s
Wall time: 20.7 s


In [401]:
%%time
uniq_rxns = df.index.unique()

CPU times: total: 6.12 s
Wall time: 6.14 s


In [402]:
import random

In [408]:
%%time
df.loc[random.choice(uniq_rxns)]

CPU times: total: 31.2 ms
Wall time: 1 ms


,trivial,name,smiles,rxn_role
rxn_id,,,,
ord-4114c6805913475585401d0f792cfa60,None,lithium aluminium hydride,[H-].[Al+3].[Li+].[H-].[H-].[H-],1
ord-4114c6805913475585401d0f792cfa60,None,ether,CCOCC,3
ord-4114c6805913475585401d0f792cfa60,amine,"4-Aminomethyl-4-benzylthio-2,3,5,6-tetrahydro-...",NCC1(CCOCC1)SCC1=CC=CC=C1,8
ord-4114c6805913475585401d0f792cfa60,None,"4-benzylthio-2,3,5,6-tetrahydro-4-nitromethyl-...",C(C1=CC=CC=C1)SC1(CCOCC1)C[N+](=O)[O-],1
ord-4114c6805913475585401d0f792cfa60,None,ether,CCOCC,3


In [337]:
x_len = 500000
g_len = 200

X = np.arange(x_len * 2).reshape(x_len, 2)
groups = np.random.randint(0, g_len, x_len)

In [371]:
%%time
rxn_ids = cmpd_np[:, 4]
uniq_rxns = np.unique(cmpd_np[:, 4])
is_product = cmpd_np[:, 3] == 8
# np.array([(is_product[rxn_ids==id]).sum() for id in rxns[:10]])

# np.bincount(uniq_rxns, is_product, len(rxns))

CPU times: total: 16.5 s
Wall time: 16.5 s
